# Pound Drever Hall

## 0.0 Introduction

Pound Drever Hall is a technique to extract frequency offset information from a laser beam. This notebook is designed to give a nice visual introduction to PDH before diving deeper into some of the key math. 

## 1.0 Sinusoid Modulation

The first thing we will do is create a time-domain sinusoid, this will represent our laser beam. We then modulate this sinusoid via an electro-optic modulator (EOM). The EOM will periodically phase-shift the signal at some given modulation frequency. Both the modulated and unmodulated carrier waves are visible below. 

In [39]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

# Generate time axis
t = np.linspace(0, 1, 1000)

def plot_sinusoid(freq=1.0):
    omega = 2 * np.pi * freq
    y = np.cos(omega * t) 
    plt.figure(figsize=(8,4))
    plt.plot(t, y, label=fr'$\cos(2\pi \omega t),\ f={freq:.2f}$ Hz')
    plt.xlabel("Time (s)")
    plt.ylabel("Amplitude")
    plt.title("Unmodulated Carrier")
    plt.grid(True)
    plt.legend()
    plt.show()


def plot_modulated_sinusoid(freq=1.0, mod_freq = 0.1, beta=0.5):
    omega = 2 * np.pi * freq
    y = np.cos(omega * t + beta * np.cos(mod_freq * t))
    plt.figure(figsize=(8,4))
    plt.plot(t, y, label=fr'$\cos(2\pi \omega t),\ f={freq:.2f}$ Hz')
    plt.xlabel("Time (s)")
    plt.ylabel("Amplitude")
    plt.title("Modulated Carrier")
    plt.grid(True)
    plt.legend()
    plt.show()


def spectrum_modulated_sinusoid(freq=1.0, mod_freq=0.1, beta=0.5):
    T = 10          
    N = 10000      
    t = np.linspace(0, T, N)
    omega_c = 2 * np.pi * freq
    omega_m = 2 * np.pi * mod_freq
    y = np.cos(omega_c * t + beta * np.cos(omega_m * t))
    Y = np.fft.fftshift(np.fft.fft(y))
    f = np.fft.fftshift(np.fft.fftfreq(N, d=t[1]-t[0]))
    Y_mag = np.abs(Y) / np.max(np.abs(Y))
    plt.figure(figsize=(10,5))
    plt.plot(f, Y_mag)
    plt.title("Frequency Spectrum of Phase-Modulated Signal")
    plt.xlabel("Frequency (Hz)")
    plt.ylabel("Normalized Magnitude")
    plt.xlim(freq-5*mod_freq, freq+5*mod_freq)
    plt.grid(True)
    plt.show()


omega=FloatSlider(value=1.0, min=0.1, max=10.0, step=0.1)
mod_freq=FloatSlider(value=1.0, min=0.1, max=10.0, step=0.1)
beta=FloatSlider(value=1.0, min=0.1, max=10.0, step=0.1)


# Interactive slider for frequency (0.1 Hz to 10 Hz for visibility)
interact(plot_sinusoid, freq=omega)
interact(plot_modulated_sinusoid, freq=omega, mod_freq=mod_freq, beta=beta)



interactive(children=(FloatSlider(value=1.0, description='freq', max=10.0, min=0.1), Output()), _dom_classes=(…

interactive(children=(FloatSlider(value=1.0, description='freq', max=10.0, min=0.1), FloatSlider(value=1.0, de…

<function __main__.plot_modulated_sinusoid(freq=1.0, mod_freq=0.1, beta=0.5)>

If we look at the modulated signal we see something that seems similar to multiple sinusoidal signals with different frequencies superimposing themselves on each other. This can be proven with the Jacobi-Anger identity but for now we'll just grab the frequency spectrum and see what happens. 

In [41]:
interact(spectrum_modulated_sinusoid, 
         freq=omega, 
         mod_freq=mod_freq, 
         beta=FloatSlider(value=0.5, min=0.0, max=3.0, step=0.1))

interactive(children=(FloatSlider(value=8.3, description='freq', max=10.0, min=0.1), FloatSlider(value=6.80000…

<function __main__.spectrum_modulated_sinusoid(freq=1.0, mod_freq=0.1, beta=0.5)>

We can see that for modulation frequencies less than the carrier frequency, and for comparatively small values of beta, we generally observe two prominent 'sidebands' equidistant on each side of the dominant band (which as you'd expect is the carrier frequency). For now we can assume that a properly designed PDH system will ensure that the modulation frequency is much lower than the carrier frequency and that our beta is fairly small.

## 2.0 Cavity interference
We can make the assumption that our modulating signal is very resistant to frequency drift. If we could find a way to translate frequency offset into a phase offset, we could then get a beat signal by superimposing the two fields. We could then demodulate that beat signal with respect to our modulating tone to get an error signal. If the beat signal is odd-symmetric around our modulating tone, demodulating it with respect to that tone will allow us to obtain the magnitude and sign of our frequency offset. In other words, we should:

1. Impart a phase-shift on a signal with respect to frequency drift that is odd-symmetric
2. Make the phase shift as sharp as possible so we can detect it easily
3. Center it on our target carrier frequency that we are trying to correct to

We accomplish the three objectives above through either a Fabri-Perot cavity or a ring resonator. For what we care about they are functionally the same. We rotate the carrier with respect to frequency offset and demodulate the reflected field with respect to the modulation frequency.

. 